In [10]:
import torch
device = torch.device('cuda:0')
torch.cuda.set_device(0)


In [1]:
import os
import time
import sys
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, LambdaLR
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from sklearn.metrics import jaccard_score # For IoU
from tqdm import tqdm
import wandb

config_path = '/home/stud/fmarchetto/SegmentationTests/Project/'  
sys.path.append(config_path)
from config import Config
wandb.login()



wandb: Currently logged in as: marchettofrancesco (marchetto_francesco) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
config_path = '/home/stud/fmarchetto/SegmentationTests/Project/'  
sys.path.append(config_path)
from config import Config
try:
    from config import Config as cfg  
    print("Config imported successfully as 'cfg'.")
except ImportError as e:
    print(f"Import failed: {e}. Check if config.py exists in {config_path} and paths match.")
cfg

Config imported successfully as 'cfg'.


{'base_dir': '/home/stud/fmarchetto/SegmentationTests/Data/',
 'datasets': {'balanced': '/home/stud/fmarchetto/SegmentationTests/Data/balanced_train_df.csv'},
 'val_csv': '/home/stud/fmarchetto/SegmentationTests/Data/val_df.csv',
 'test_csv': '/home/stud/fmarchetto/SegmentationTests/Data/test_df.csv',
 'models': [{'name': 'U-Net ResNet34', 'arch': 'Unet', 'encoder': 'resnet34'},
  {'name': 'U-Net efficientnet-b2',
   'arch': 'Unet',
   'encoder': 'efficientnet-b2'}],
 'batch_sizes': [8, 16],
 'learning_rates': [0.001, 0.0001],
 'optimizers': [{'name': 'SGD_Momentum',
   'class': 'SGD',
   'extra_params': {'momentum': 0.9, 'nesterov': True}},
  {'name': 'RMSprop', 'class': 'RMSprop', 'extra_params': {'momentum': 0.9}},
  {'name': 'Adam', 'class': 'Adam'}],
 'epochs': 8,
 'patience_early_stop': 3,
 'device': "torch.device('cuda' if torch.cuda.is_available() else 'cpu')",
 'wandb_project': 'Zuliani1_Marchetto2',
 'transform': 'A.Compose([\n    A.Rotate(limit=10, p=0.5), A.ShiftScaleRotate

In [ ]:

device = eval(cfg['device'])
print(f"\n✓ Selected device: {device}")
print(f"✓ Current device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")

In [12]:
!pwd
!ls -l /home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/TCGA_DU_6408_19860521/TCGA_DU_6408_19860521_46.tif
!ls -l /home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/TCGA_CS_6290_20000917/TCGA_CS_6290_20000917_9_mask.tif

/home/stud/fmarchetto/SegmentationTests/Project
ls: cannot access '/home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/TCGA_DU_6408_19860521/TCGA_DU_6408_19860521_46.tif': No such file or directory
-rw-r--r-- 1 fmarchetto stud 65746 Oct 31 10:45 /home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/TCGA_CS_6290_20000917/TCGA_CS_6290_20000917_9_mask.tif


Next is the code we used to train all the combinations of 2 models, 'U-Net ResNet34'and 'efficentnet_b2', with 2 different learning rates, we used 4 optimizers, we started with SGD and ADAM, then used SGD with momentum and RMSprop, 2 different batch sizes 8,16 for 8 epochs

In [13]:

from sklearn.model_selection import train_test_split

data_dir = '/home/stud/fmarchetto/SegmentationTests/Data/kaggle_3m/'
IMG_SIZE = 128

# --- Collect all image/mask paths ---
image_paths = []
mask_paths = []

print("Collecting image and mask paths...")
for case_folder in os.listdir(data_dir):
    case_path = os.path.join(data_dir, case_folder)
    if not os.path.isdir(case_path):
        continue
    for file in os.listdir(case_path):
        if file.endswith('.tif') and not file.endswith('_mask.tif'):
            image_paths.append(os.path.join(case_path, file))
            mask_file = file.replace('.tif', '_mask.tif')
            mask_paths.append(os.path.join(case_path, mask_file))

print(f"Found {len(image_paths)} images")

# --- Create DataFrame ---
df_full = pd.DataFrame({
    "image_path": image_paths,
    "mask_path": mask_paths
})

# --- Exact same preprocessing function as your working code ---
def process_image(image_path, mask_path, img_size=IMG_SIZE):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    image = cv2.resize(image, (img_size, img_size))
    mask = cv2.resize(mask, (img_size, img_size))
    
    image = image / 255.0
    mask = mask / 255.0
    mask = np.where(mask > 0.5, 1, 0)  # Binary
    
    return image.astype(np.float32), mask.astype(np.float32).reshape(img_size, img_size, 1)

# --- Preprocess all data ---
print("Preprocessing all images (this takes ~30-60 seconds)...")
images = []
masks = []

for img_path, msk_path in tqdm(zip(df_full["image_path"], df_full["mask_path"]), total=len(df_full)):
    img, msk = process_image(img_path, msk_path)
    images.append(img)
    masks.append(msk)

images = np.array(images)   # [N, 128, 128, 3]
masks = np.array(masks)     # [N, 128, 128, 1]

print(f"Final shapes → Images: {images.shape}, Masks: {masks.shape}")

# --- 80/10/10 split  ---
X_train, X_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.2, random_state=42)
X_val,   X_test, y_val,   y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

train_df = pd.DataFrame({"dummy_path": range(len(X_train))})
val_df   = pd.DataFrame({"dummy_path": range(len(X_val))})

GLOBAL_DATA = {
    "train_images": X_train,
    "train_masks":  y_train,
    "val_images":   X_val,
    "val_masks":    y_val,
    "test_images":  X_test,
    "test_masks":   y_test
}

class MriDataset(Dataset):
    def __init__(self, df, transform=None, img_size=128, mode="train"):
        self.df = df
        self.transform = transform
        self.img_size = img_size
        self.mode = mode  # "train", "val", or "test"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        if self.mode == "train":
            img = GLOBAL_DATA["train_images"][idx]
            mask = GLOBAL_DATA["train_masks"][idx]
        elif self.mode == "val":
            img = GLOBAL_DATA["val_images"][idx]
            mask = GLOBAL_DATA["val_masks"][idx]
        else:
            img = GLOBAL_DATA["test_images"][idx]
            mask = GLOBAL_DATA["test_masks"][idx]

        img = img.copy()
        mask = mask.copy()

        # Albumentations expects H W C numpy arrays
        img_np = (img * 255).astype(np.uint8)           # back to 0-255
        mask_np = (mask.squeeze() * 255).astype(np.uint8)  # back to 0/255

        if self.transform:
            augmented = self.transform(image=img_np, mask=mask_np)
            img = augmented['image']      # [C, H, W] tensor, 0-255
            mask = augmented['mask']      # [H, W] tensor

            # Normalize image to [0,1]
            if img.dtype == torch.uint8:
                img = img.float() / 255.0
            elif img.max() > 1.0:
                img = img / 255.0

            # Fix mask
            if mask.ndim == 2:
                mask = mask.unsqueeze(0)
            mask = mask.float() / 255.0
            mask = (mask > 0.5).float()
        else:
            img = torch.from_numpy(img.transpose(2, 0, 1))
            mask = torch.from_numpy(mask.transpose(2, 0, 1)).float()

        return img, mask
# Loss and Metrics (Dice loss) 

class BCE_Dice_Loss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, pred, target):
        
        if target.shape[1] != 1:  # If target is [batch, H, W, 1], permute to [batch, 1, H, W]
            target = target.permute(0, 3, 1, 2)
        elif target.ndim == 3:  # If [batch, H, W], unsqueeze to [batch, 1, H, W]
            target = target.unsqueeze(1)
        
        bce = self.bce(pred, target)
        pred_sig = torch.sigmoid(pred)
        intersection = (pred_sig * target).sum(dim=(2,3))
        dice = (2 * intersection + 1e-6) / (pred_sig.sum(dim=(2,3)) + target.sum(dim=(2,3)) + 1e-6)
        dice_loss = 1 - dice.mean()
        return bce + dice_loss
        

def iou_coef(y_true, y_pred, threshold=0.5, smooth=1):
    """
    y_true: [B, 1, H, W] float tensor
    y_pred: [B, 1, H, W] logits
    """
    y_pred = torch.sigmoid(y_pred)
    y_pred = (y_pred > threshold).float()
    
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    
    return iou.item()  # Return scalar
    
def dice_coef(y_true, y_pred, threshold=0.5, smooth=1):
    y_pred = torch.sigmoid(y_pred)
    y_pred = (y_pred > threshold).float()
    
    y_true = y_true.view(-1)
    y_pred = y_pred.view(-1)
    
    intersection = (y_true * y_pred).sum()
    return (2. * intersection + smooth) / (y_true.sum() + y_pred.sum() + smooth).item()

def compute_class_iou(pred, target, threshold=0.5, class_id=1):
    pred_prob = torch.sigmoid(pred)  # ← ADD THIS
    pred_bin = (pred_prob > threshold).float().cpu().numpy().flatten()
    target = (target == class_id).float().cpu().numpy().flatten()
    return jaccard_score(target, pred_bin, average='binary', zero_division=0)

# --- Early Stopping Class ---
class EarlyStopping:
    def __init__(self, patience=3, delta=0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            


def train_model(dataset_type, batch_size, lr, optimizer_name, model_info):
    # Load datasets
    train_csv = cfg['datasets'][dataset_type]
    train_df = pd.read_csv(train_csv)
    val_df = pd.read_csv(cfg['val_csv'])
    
    transform = A.Compose([
    A.Rotate(limit=10, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=0, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    ToTensorV2()  # This converts image to float32 tensor, 0-255 range
    ])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Don't eval this
    
   
    train_dataset = MriDataset(pd.DataFrame({"idx": range(len(X_train))}), transform=transform, mode="train")
    val_dataset   = MriDataset(pd.DataFrame({"idx": range(len(X_val))}),   transform=transform,   mode="val")
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    # Model setup
    model = getattr(smp, model_info['arch'])(encoder_name=model_info['encoder'], classes=1, encoder_weights='imagenet', activation=None).to(device)
    
    opt_config = next(opt for opt in cfg['optimizers'] if opt['name'] == optimizer_name)
    optimizer_class = getattr(torch.optim, opt_config['class'])
    extra_params = opt_config.get('extra_params', {})
    optimizer = optimizer_class(model.parameters(), lr=lr, **extra_params)
    
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=1, factor=0.7)  
    warmup_epochs = 3
    warmup_scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: min((epoch + 1) / warmup_epochs, 1.0))
    
    run_name = f"{model_info['name']}_{optimizer_name}_LR{lr}_Batch{batch_size}_{dataset_type}"
    run = wandb.init(
        project=cfg['wandb_project'],
        name=run_name,
        config={
            'dataset_type': dataset_type,
            'batch_size': batch_size,
            'lr': lr,
            'optimizer': optimizer_name,
            'model': model_info['name'],
            'epochs': cfg['epochs']
        }
    )
    
    early_stopping = EarlyStopping(patience=cfg['patience_early_stop'])
    best_model_path = f"best_model_{dataset_type}_{model_info['name']}.pth"
    loss_fn = BCE_Dice_Loss()

    for epoch in range(cfg['epochs']):
        model.train()
        train_loss = 0
        train_dice = 0
        train_iou = 0
        train_acc = 0
        num_batches = len(train_loader)
        
        for img, mask in tqdm(train_loader, desc=f"Epoch {epoch+1}/{cfg['epochs']} - Train"):
            img, mask = img.to(device), mask.to(device)
            optimizer.zero_grad()
            pred = model(img)
            loss = loss_fn(pred, mask)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            train_dice += dice_coef(mask, pred)
            train_iou += iou_coef(mask, pred)
            pred_prob = torch.sigmoid(pred)  
            pred_bin = (pred_prob > 0.5).float()
            train_acc += (pred_bin == mask).float().mean().item()
        
        train_loss /= num_batches
        train_dice /= num_batches
        train_iou /= num_batches
        train_iou_tumor /= num_batches
        train_acc /= num_batches
        
        # Validation
        model.eval()
        val_loss = 0
        val_dice = 0
        val_iou = 0
        val_acc = 0
        num_val_batches = len(val_loader)
        
        with torch.no_grad():
            for img, mask in tqdm(val_loader, desc=f"Epoch {epoch+1}/{cfg['epochs']} - Val"):
                img, mask = img.to(device), mask.to(device)
                pred = model(img)
                val_loss += loss_fn(pred, mask).item()
                val_dice += dice_coef(mask, pred)
                val_iou += iou_coef(mask, pred)
                pred_prob = torch.sigmoid(pred) 
                pred_bin = (pred_prob > 0.5).float()
                val_acc += (pred_bin == mask).float().mean().item()
        
        val_loss /= num_val_batches
        val_dice /= num_val_batches
        val_iou /= num_val_batches
        val_acc /= num_val_batches
        
        if epoch < warmup_epochs:
            warmup_scheduler.step()
        scheduler.step(val_loss)
        
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break
        
        # Log to WandB 
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_dice': train_dice,
            'val_dice': val_dice,
            'train_iou': train_iou,
            'val_iou': val_iou,
            'train_acc': train_acc,
            'val_acc': val_acc
        })
    
    # Save model
    torch.save(model.state_dict(), best_model_path)

    run.finish()  # End wandb run

# --- Run the Grid Search Over All Combinations ---
for dataset_type in cfg['datasets'].keys():
    for batch_size in cfg['batch_sizes']:
        for lr in cfg['learning_rates']:
            for opt in cfg['optimizers']:
                for model_info in cfg['models']:
                    print(f"\nStarting run: Dataset={dataset_type}, Batch={batch_size}, LR={lr}, Opt={opt['name']}, Model={model_info['name']}")
                    train_model(dataset_type, batch_size, lr, opt['name'], model_info)

Found 3929 images
Preprocessing all images (this takes ~30-60 seconds)...


100%|█████████████████████████████████████████████████████████| 3929/3929 [00:14<00:00, 261.95it/s]


Final shapes → Images: (3929, 128, 128, 3), Masks: (3929, 128, 128, 1)
Train: (3143, 128, 128, 3), Val: (393, 128, 128, 3), Test: (393, 128, 128, 3)

Starting run: Dataset=balanced, Batch=8, LR=0.001, Opt=SGD_Momentum, Model=U-Net ResNet34


/home/stud/fmarchetto/myenv/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 8/8 - Val: 100%|█████████████████████████████████████████████| 50/50 [00:02<00:00, 24.46it/s]


...

Epoch 8/8 - Val: 100%|█████████████████████████████████████████████| 50/50 [00:02<00:00, 23.72it/s]


epoch,▁▂▃▄▅▆▇█
train_acc,▁▃▆▇▇███
train_dice,▁▁▂▂▃▄▆█
train_iou,▁▁▂▂▃▄▆█
train_loss,█▇▅▄▃▃▂▁
val_acc,▁▄▆▇████
val_dice,▁▁▂▂▃▅▇█
val_iou,▁▁▂▂▃▅▆█
val_loss,█▇▅▄▃▃▂▁
epoch,8
train_acc,0.97249



Starting run: Dataset=balanced, Batch=8, LR=0.0001, Opt=SGD_Momentum, Model=U-Net efficientnet-b2


/home/stud/fmarchetto/myenv/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 4/8 - Train:  81%|█████████████████████████████████▏       | 318/393 [00:29<00:06, 10.77it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

